In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.GPBO_Class_fxns import * #Fix this later

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.21 s, sys: 274 ms, total: 2.48 s
Wall time: 2.52 s


## Define Method and DateTime

In [2]:
#Generate Method (except last part)
meth_name = Method_name_enum(1)
method = GPBO_Methods(meth_name)

print("Method Name: ", method.method_name.name)
print("Emulator?: ", method.emulator)
print("Obj Func: ", method.obj.name)
print("Sparse Grid?: ", method.sparse_grid)

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

Method Name:  A1
Emulator?:  False
Obj Func:  OBJ
Sparse Grid?:  False


## Initialize Simulator

In [3]:
def simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, case_study_parameters):
    """
    Sets the model for calculating y based off of the case study identifier.

    Parameters
    ----------
    cs_name: Class, The name/enumerator associated with the case study being evaluated

    Returns
    -------
    calc_y_fxn: function, the function used for calculation is case study cs_name.name
    """
    #Note: Add your function name from GPBO_Class_fxns.py here
    if cs_name.value == 1:      
        theta_ref = np.array([1.0, -1.0])
        theta_names = ['theta_1', 'theta_2']
        bounds_x_l = [-2]
        bounds_x_u = [2]
        bounds_theta_l = [-2, -2]
        bounds_theta_u = [ 2,  2]
        calc_y_fxn = calc_cs1_polynomial
        
    elif cs_name.value == 2:     
        theta_ref = np.array([-200,-100,-170,15,-1,-1,-6.5,0.7,0,0,11,0.6,-10,-10,-6.5,0.7,1,0,-0.5,-1,0,0.5,1.5,1])
                             
        theta_names = ['A_1', 'A_2', 'A_3', 'A_4', 'a_1', 'a_2', 'a_3', 'a_4', 'b_1', 'b_2', 'b_3', 'b_4', 'c_1', 
                       'c_2', 'c_3', 'c_4', 'x0_1', 'x0_2', 'x0_3', 'x0_4', 'x1_1', 'x1_2', 'x1_3', 'x1_4']
        bounds_x_l = [-1.5, -0.5]
        bounds_x_u = [1, 2]
        bounds_theta_l = [-300,-200,-250, 5,-2,-2,-10, -2, -2,-2,5,-2,-20,-20, -10,-1 ,-2,-2,-2, -2,-2,-2,0,-2]
        bounds_theta_u = [-100,  0, -150, 20,2, 2, 0,  2,  2,  2, 15,2, 0,0   , 0,  2, 2,  2, 2, 2 ,2 , 2, 2,2]
        calc_y_fxn = calc_muller
        
    else:
        raise ValueError("self.CaseStudyParameters.cs_name.value must exist!")

    return Simulator(indecies_to_consider, 
                     theta_ref,
                     theta_names,
                     bounds_theta_l, 
                     bounds_x_l, 
                     bounds_theta_u, 
                     bounds_x_u, 
                     noise_mean,
                     noise_std,
                     case_study_parameters,
                     calc_y_fxn)

## Initialize Case Study and Simulator

In [4]:
cs_name  = CS_name_enum(1)
print("CS Name: ", cs_name.name)

# indecies_to_consider = list(range(4, 12)) #This is what changes for different subproblems of CS2
indecies_to_consider = list(range(0, 2)) #This is what changes for different subproblems of CS1

ep0 = 1
sep_fact = 0.8
normalize = False
lhs_gen_theta = True
eval_all_pairs = False
noise_mean = 0
noise_std = 0.01
noise_std = 0
kernel = Kernel_enum(1)
set_lenscl = 1
outputscl = False
retrain_GP = 2
GP_train_iter = 300
bo_iter_tot = 3
bo_run_tot = 2
save_fig = False
save_data = False
num_data = None
seed = 1

cs_params = CaseStudyParameters(cs_name, ep0, sep_fact, normalize, eval_all_pairs, bo_iter_tot, bo_run_tot, 
                         save_fig, save_data, DateTime, seed)

simulator = simulator_helper_test_fxns(cs_name, indecies_to_consider, noise_mean, noise_std, cs_params)

CS Name:  CS1


## Test Simulator

In [5]:
#Test get_sim_num_theta_data
num_theta_data = 20
gen_meth_theta = Gen_meth_enum(1)
sim_num_theta = simulator.get_sim_num_theta_data(num_theta_data, gen_meth_theta)
print(sim_num_theta == num_theta_data)

gen_meth_theta = Gen_meth_enum(2)
sim_num_theta = simulator.get_sim_num_theta_data(num_theta_data, gen_meth_theta)
print(sim_num_theta == num_theta_data**simulator.dim_theta)

True
True


In [6]:
#Test __set_true_params
print(simulator.theta_true, simulator.theta_true_names)

[ 1. -1.] ['theta_1', 'theta_2']


In [7]:
#Test gen_exp_data
num_x_data = 5

gen_meth_x = Gen_meth_enum(1)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
print(len(exp_data.theta_vals) == len(exp_data.x_vals) == len(exp_data.y_vals))

gen_meth_x = Gen_meth_enum(2)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
print(len(exp_data.theta_vals) == len(exp_data.x_vals) == len(exp_data.y_vals))

True
True


In [8]:
#Test gen_sim_data
gen_meth_x = Gen_meth_enum(1)
gen_meth_theta = Gen_meth_enum(1)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
print(len(sim_data.theta_vals), len(sim_data.y_vals))

gen_meth_x = Gen_meth_enum(2)
gen_meth_theta = Gen_meth_enum(1)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
print(len(sim_data.theta_vals), len(sim_data.y_vals))

gen_meth_x = Gen_meth_enum(1)
gen_meth_theta = Gen_meth_enum(2)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
print(len(sim_data.theta_vals), len(sim_data.y_vals))

gen_meth_x = Gen_meth_enum(2)
gen_meth_theta = Gen_meth_enum(2)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
print(len(sim_data.y_vals))

100 100
100 100
2000 2000
2000


In [9]:
#Test sim_data_to_sse_sim_data
gen_meth_x = Gen_meth_enum(1)
gen_meth_theta = Gen_meth_enum(1)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
sse_sim_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data)
print(sse_sim_data.y_vals.shape)

gen_meth_x = Gen_meth_enum(2)
gen_meth_theta = Gen_meth_enum(1)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
sse_sim_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data)
print(sse_sim_data.y_vals.shape)

gen_meth_x = Gen_meth_enum(1)
gen_meth_theta = Gen_meth_enum(2)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
sse_sim_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data)
print(sse_sim_data.y_vals.shape)

gen_meth_x = Gen_meth_enum(2)
gen_meth_theta = Gen_meth_enum(2)
sim_data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)
sse_sim_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data)
print(sse_sim_data.y_vals.shape)
print(sse_sim_data.theta_vals[np.argmin(sse_sim_data.y_vals)], np.min(sse_sim_data.y_vals))

(20,)
(20,)
(400,)
(400,)
[ 0.94736842 -0.94736842] 0.1218836565096953


In [10]:
#Test gen_y_data
data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
y_data = simulator.gen_y_data(data, noise_mean, noise_std)
print(len(y_data))

data = simulator.gen_sim_data(method, num_theta_data, num_x_data, gen_meth_theta, gen_meth_x)
y_data = simulator.gen_y_data(data, 0, 0)
print(len(y_data))

2000
2000


In [11]:
#Test __lhs_sampling(self, num_points, bounds, seed)
num_points = 5
bounds = np.array([[0,0],[1,1]])
seed = 1
lhs = simulator._Simulator__lhs_sampling(num_points, bounds, seed)
print(lhs)

[[0.29763568 0.80990726]
 [0.77116808 0.61027011]
 [0.13763371 0.31533471]
 [0.43445948 0.11816017]
 [0.89008126 0.59448818]]


In [12]:
#Test _grid_sampling(self, num_points, bounds)
num_points = 5
bounds = np.array([[0,0],[1,1]])
lhs = simulator._Simulator__grid_sampling(num_points, bounds)
print(lhs)

[[0.   0.  ]
 [0.   0.25]
 [0.   0.5 ]
 [0.   0.75]
 [0.   1.  ]
 [0.25 0.  ]
 [0.25 0.25]
 [0.25 0.5 ]
 [0.25 0.75]
 [0.25 1.  ]
 [0.5  0.  ]
 [0.5  0.25]
 [0.5  0.5 ]
 [0.5  0.75]
 [0.5  1.  ]
 [0.75 0.  ]
 [0.75 0.25]
 [0.75 0.5 ]
 [0.75 0.75]
 [0.75 1.  ]
 [1.   0.  ]
 [1.   0.25]
 [1.   0.5 ]
 [1.   0.75]
 [1.   1.  ]]


In [13]:
#Test __create_param_data(self, num_points, bounds, gen_meth)
num_points, bounds, gen_meth = 5, np.array([[0,0],[1,1]]), Gen_meth_enum(1)
data = simulator._Simulator__create_param_data(num_points, bounds, gen_meth)
print(data)
num_points, bounds, gen_meth = 5, np.array([[0,0],[1,1]]), Gen_meth_enum(2)
data = simulator._Simulator__create_param_data(num_points, bounds, gen_meth)
print(data)

[[0.29763568 0.80990726]
 [0.77116808 0.61027011]
 [0.13763371 0.31533471]
 [0.43445948 0.11816017]
 [0.89008126 0.59448818]]
[[0.   0.  ]
 [0.   0.25]
 [0.   0.5 ]
 [0.   0.75]
 [0.   1.  ]
 [0.25 0.  ]
 [0.25 0.25]
 [0.25 0.5 ]
 [0.25 0.75]
 [0.25 1.  ]
 [0.5  0.  ]
 [0.5  0.25]
 [0.5  0.5 ]
 [0.5  0.75]
 [0.5  1.  ]
 [0.75 0.  ]
 [0.75 0.25]
 [0.75 0.5 ]
 [0.75 0.75]
 [0.75 1.  ]
 [1.   0.  ]
 [1.   0.25]
 [1.   0.5 ]
 [1.   0.75]
 [1.   1.  ]]
